<a href="https://colab.research.google.com/github/shubhamshuklaa/deep_learning/blob/main/CNN_OPTIMIZE_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CREATE CNN MODEL AND OPTIMIZE IT USING KERAS

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images.shape

(60000, 28, 28)

In [9]:
#reshaping the dataset
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
#building the model
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)
        
      ),
          keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'
          ),
          keras.layers.Flatten(),
          keras.layers.Dense(
              units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
              activation='relu'
          ),
          keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [11]:
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.0 MB/s eta 0:00:00


In [12]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9028333425521851

Best val_accuracy So Far: 0.9131666421890259
Total elapsed time: 00h 02m 40s


In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 64)        1664      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 80)                2478160   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 2,517,562
Trainable params: 2,517,562
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1522 - accuracy: 0.9428 - val_loss: 0.2556 - val_accuracy: 0.9127
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1127 - accuracy: 0.9586 - val_loss: 0.2604 - val_accuracy: 0.9113
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0823 - accuracy: 0.9699 - val_loss: 0.2881 - val_accuracy: 0.9140
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0630 - accuracy: 0.9770 - val_loss: 0.3826 - val_accuracy: 0.9148
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0480 - accuracy: 0.9826 - val_loss: 0.4021 - val_accuracy: 0.9132
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0391 - accuracy: 0.9854 - val_loss: 0.4467 - val_accuracy: 0.9073
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0327 - accuracy: 0.9884 - val_loss: 0.4738 - val_accuracy